# PRISMA Search Strategy Nanopublication Creator

Creates Search Strategy nanopublications from a JSON configuration file.

**Template:** [Systematic Review Search Strategy](https://w3id.org/np/RAvcJKm2DZPEKOBevGdPGcKL6sEw04JXwgzx2lH5DE5LU)

---

## Instructions

1. **Create a JSON file** with your search strategy details (see template below)
2. **Set the path** to your JSON file in Section 1
3. **Run All Cells** → Get your `.trig` file

---
# 📁 SECTION 1: INPUT FILE (EDIT THIS)
---

In [1]:
# Path to your Search Strategy JSON file
STRATEGY_FILE = "../inputs/quantum-biodiversity/search-strategy-quantum-biodiversity.json"

---
# ⚙️ SECTION 2: SETUP
---

In [2]:
# Install dependencies (uncomment if needed)
# !pip install nanopub rdflib

In [3]:
import json
from rdflib import Graph, Dataset, Namespace, Literal, URIRef
from rdflib.namespace import RDF, RDFS, XSD, FOAF
from datetime import datetime, timezone
from pathlib import Path

# Namespaces (matching Nanodash)
NP = Namespace("http://www.nanopub.org/nschema#")
DCT = Namespace("http://purl.org/dc/terms/")
NT = Namespace("https://w3id.org/np/o/ntemplate/")
NPX = Namespace("http://purl.org/nanopub/x/")
PROV = Namespace("http://www.w3.org/ns/prov#")
ORCID = Namespace("https://orcid.org/")
DCAT = Namespace("http://www.w3.org/ns/dcat#")
SLV = Namespace("https://w3id.org/sciencelive/o/terms/")

# Search Strategy template URIs
SEARCH_TEMPLATE = URIRef("https://w3id.org/np/RAvcJKm2DZPEKOBevGdPGcKL6sEw04JXwgzx2lH5DE5LU")
PROV_TEMPLATE = URIRef("https://w3id.org/np/RA7lSq6MuK_TIC6JMSHvLtee3lpLoZDOqLJCLXevnrPoU")
PUBINFO_TEMPLATE_1 = URIRef("https://w3id.org/np/RA0J4vUn_dekg-U1kK3AOEt02p9mT2WO03uGxLDec1jLw")
PUBINFO_TEMPLATE_2 = URIRef("https://w3id.org/np/RAukAcWHRDlkqxk7H2XNSegc1WnHI569INvNr-xdptDGI")

print("✓ Setup complete")

✓ Setup complete


---
# 📖 SECTION 3: LOAD & VALIDATE
---

In [4]:
# Load Search Strategy from JSON
print(f"Loading: {STRATEGY_FILE}")

with open(STRATEGY_FILE, 'r', encoding='utf-8') as f:
    config = json.load(f)

# Extract fields
AUTHOR_ORCID = config['author']['orcid']
AUTHOR_NAME = config['author']['name']

LABEL = config['search_strategy']['label']
PART_OF = config['search_strategy'].get('part_of')  # Optional URI to PICO nanopub
DATABASES = config['search_strategy']['databases']  # List of URIs
SEARCH_TERMS = config['search_strategy']['search_terms']  # List of URIs
START_DATE = config['search_strategy']['time_period']['start']
END_DATE = config['search_strategy']['time_period']['end']
LANGUAGES = config['search_strategy'].get('languages', [])  # Optional list of URIs
METHODOLOGY_NOTES = config['search_strategy'].get('methodology_notes', '')  # Optional

OUTPUT_FILENAME = config['output']['filename']

print(f"✓ Loaded: {LABEL[:50]}...")

Loading: search-strategy-quantum-biodiversity.json
✓ Loaded: Quantum Computing for Biodiversity - Literature Se...


In [5]:
# Validate
print("Validating...")

errors = []
if not AUTHOR_ORCID:
    errors.append("author.orcid is required")
if not AUTHOR_NAME:
    errors.append("author.name is required")
if not LABEL or len(LABEL) < 10:
    errors.append("search_strategy.label must be at least 10 characters")
if not DATABASES or len(DATABASES) == 0:
    errors.append("search_strategy.databases must have at least one database")
if not SEARCH_TERMS or len(SEARCH_TERMS) == 0:
    errors.append("search_strategy.search_terms must have at least one term")
if not START_DATE:
    errors.append("search_strategy.time_period.start is required")
if not END_DATE:
    errors.append("search_strategy.time_period.end is required")

if errors:
    print("❌ Validation errors:")
    for e in errors:
        print(f"   - {e}")
    raise ValueError("Please fix the errors in your JSON file")
else:
    print("✓ All fields valid")

Validating...
✓ All fields valid


---
# 🔨 SECTION 4: BUILD NANOPUBLICATION
---

In [6]:
# Create dataset with named graphs
TEMP_NP = Namespace("http://purl.org/nanopub/temp/np/")

this_np = URIRef("http://purl.org/nanopub/temp/np/")
head_graph = URIRef("http://purl.org/nanopub/temp/np/Head")
assertion_graph = URIRef("http://purl.org/nanopub/temp/np/assertion")
provenance_graph = URIRef("http://purl.org/nanopub/temp/np/provenance")
pubinfo_graph = URIRef("http://purl.org/nanopub/temp/np/pubinfo")

# Subject URIs
search_strategy_uri = TEMP_NP["searchStrategy"]
time_period_uri = TEMP_NP["timePeriod"]
author_uri = ORCID[AUTHOR_ORCID]

ds = Dataset()

# Bind prefixes (required for nanopub library to parse correctly)
ds.bind("this", "http://purl.org/nanopub/temp/np/")
ds.bind("sub", TEMP_NP)
ds.bind("np", NP)
ds.bind("dct", DCT)
ds.bind("nt", NT)
ds.bind("npx", NPX)
ds.bind("xsd", XSD)
ds.bind("rdfs", RDFS)
ds.bind("orcid", ORCID)
ds.bind("prov", PROV)
ds.bind("foaf", FOAF)
ds.bind("dcat", DCAT)
ds.bind("slv", SLV)

print("✓ Dataset created")

✓ Dataset created


In [7]:
# HEAD graph
head = ds.graph(head_graph)
head.add((this_np, RDF.type, NP.Nanopublication))
head.add((this_np, NP.hasAssertion, assertion_graph))
head.add((this_np, NP.hasProvenance, provenance_graph))
head.add((this_np, NP.hasPublicationInfo, pubinfo_graph))

print(f"✓ Head graph: {len(head)} triples")

✓ Head graph: 4 triples


In [8]:
# ASSERTION graph
assertion = ds.graph(assertion_graph)

# Type and label
assertion.add((search_strategy_uri, RDF.type, SLV.SystematicReviewSearchStrategy))
assertion.add((search_strategy_uri, RDFS.label, Literal(LABEL)))

# Part of (link to PICO)
if PART_OF:
    assertion.add((search_strategy_uri, DCT.isPartOf, URIRef(PART_OF)))

# Databases (repeatable)
for db_uri in DATABASES:
    assertion.add((search_strategy_uri, SLV.hasSearchDatabase, URIRef(db_uri)))

# Search terms (repeatable)
for term_uri in SEARCH_TERMS:
    assertion.add((search_strategy_uri, SLV.hasSearchTerms, URIRef(term_uri)))

# Time period
assertion.add((search_strategy_uri, SLV.coversTimePeriod, time_period_uri))
assertion.add((time_period_uri, DCAT.temporalStartDate, Literal(START_DATE, datatype=XSD.date)))
assertion.add((time_period_uri, DCAT.temporalEndDate, Literal(END_DATE, datatype=XSD.date)))

# Languages (optional, repeatable)
for lang_uri in LANGUAGES:
    assertion.add((search_strategy_uri, SLV.coversLanguage, URIRef(lang_uri)))

# Methodology notes (optional)
if METHODOLOGY_NOTES:
    assertion.add((search_strategy_uri, SLV.hasMethodologyNotes, Literal(METHODOLOGY_NOTES)))

print(f"✓ Assertion graph: {len(assertion)} triples")

✓ Assertion graph: 16 triples


In [9]:
# PROVENANCE graph
provenance = ds.graph(provenance_graph)
provenance.add((assertion_graph, PROV.wasAttributedTo, author_uri))

print(f"✓ Provenance graph: {len(provenance)} triples")

✓ Provenance graph: 1 triples


In [10]:
# PUBINFO graph
pubinfo = ds.graph(pubinfo_graph)

# Truncate label for nanopub label (max ~80 chars)
np_label = f"Search Strategy: {LABEL[:60]}..." if len(LABEL) > 60 else f"Search Strategy: {LABEL}"

pubinfo.add((this_np, RDFS.label, Literal(np_label)))
pubinfo.add((this_np, DCT.creator, author_uri))
pubinfo.add((this_np, DCT.created, Literal(datetime.now(timezone.utc).isoformat(), datatype=XSD.dateTime)))
pubinfo.add((this_np, DCT.license, URIRef("https://creativecommons.org/licenses/by/4.0/")))
pubinfo.add((this_np, NPX.wasCreatedAt, URIRef("https://sciencelive4all.org/")))

# Template references
pubinfo.add((this_np, NT.wasCreatedFromTemplate, SEARCH_TEMPLATE))
pubinfo.add((this_np, NT.wasCreatedFromProvenanceTemplate, PROV_TEMPLATE))
pubinfo.add((this_np, NT.wasCreatedFromPubinfoTemplate, PUBINFO_TEMPLATE_1))
pubinfo.add((this_np, NT.wasCreatedFromPubinfoTemplate, PUBINFO_TEMPLATE_2))

# Author name
pubinfo.add((author_uri, FOAF.name, Literal(AUTHOR_NAME)))

print(f"✓ Pubinfo graph: {len(pubinfo)} triples")

✓ Pubinfo graph: 10 triples


---
# 💾 SECTION 5: SAVE OUTPUT
---

In [11]:
# Serialize and save
trig_output = ds.serialize(format='trig')

output_path = Path(f"{OUTPUT_FILENAME}.trig")
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(trig_output)

print(f"✓ Saved: {output_path}")

✓ Saved: quantum-biodiversity-search-strategy.trig


In [12]:
# Display output
print("=" * 70)
print("NANOPUBLICATION (TriG format)")
print("=" * 70)
print(trig_output)

NANOPUBLICATION (TriG format)
@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix np: <http://www.nanopub.org/nschema#> .
@prefix npx: <http://purl.org/nanopub/x/> .
@prefix nt: <https://w3id.org/np/o/ntemplate/> .
@prefix orcid: <https://orcid.org/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix slv: <https://w3id.org/sciencelive/o/terms/> .
@prefix sub: <http://purl.org/nanopub/temp/np/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

sub:provenance {
    sub:assertion prov:wasAttributedTo orcid:0000-0002-1784-2920 .
}

sub:pubinfo {
    sub: rdfs:label "Search Strategy: Quantum Computing for Biodiversity - Literature Search" ;
        dct:created "2025-12-24T18:19:41.995274+00:00"^^xsd:dateTime ;
        dct:creator orcid:0000-0002-1784-2920 ;
        dct:license <https://creativecommons.org/licenses/by/4.0/> ;
        npx:was

In [13]:
# Summary
print("=" * 70)
print("SUMMARY")
print("=" * 70)
print(f"Input:      {STRATEGY_FILE}")
print(f"Output:     {output_path}")
print(f"Author:     {AUTHOR_NAME} (orcid:{AUTHOR_ORCID})")
print()
print("Search Strategy:")
print(f"  Label:      {LABEL[:55]}..." if len(LABEL) > 55 else f"  Label:      {LABEL}")
print(f"  Databases:  {len(DATABASES)}")
print(f"  Terms:      {len(SEARCH_TERMS)}")
print(f"  Period:     {START_DATE} to {END_DATE}")
print(f"  Languages:  {len(LANGUAGES)}")
if PART_OF:
    print(f"  Part of:    {PART_OF}")
print()
print("Next steps:")
print(f"  Sign:    nanopub sign {output_path}")
print(f"  Publish: nanopub publish {output_path.stem}.signed.trig")

SUMMARY
Input:      search-strategy-quantum-biodiversity.json
Output:     quantum-biodiversity-search-strategy.trig
Author:     Anne Fouilloux (orcid:0000-0002-1784-2920)

Search Strategy:
  Label:      Quantum Computing for Biodiversity - Literature Search
  Databases:  4
  Terms:      4
  Period:     2015-01-01 to 2025-12-01
  Languages:  1
  Part of:    https://w3id.org/np/RA8B3ptXUOsN7obpkFGtA0FBmsh0OnID53wOsUIpSKTcg

Next steps:
  Sign:    nanopub sign quantum-biodiversity-search-strategy.trig
  Publish: nanopub publish quantum-biodiversity-search-strategy.signed.trig


---
# 🚀 SECTION 6: SIGN & PUBLISH (OPTIONAL)
---

In [14]:
PUBLISH = True
USE_TEST_SERVER = False

In [15]:
if PUBLISH:
    from nanopub import Nanopub, NanopubConf, load_profile
    
    profile = load_profile()
    print(f"Loaded profile: {profile.name}")
    
    conf = NanopubConf(profile=profile, use_test_server=USE_TEST_SERVER)
    np_obj = Nanopub(rdf=output_path, conf=conf)
    
    np_obj.sign()
    print(f"✓ Signed")
    
    signed_path = Path(f"{OUTPUT_FILENAME}.signed.trig")
    np_obj.store(signed_path)
    print(f"✓ Saved: {signed_path}")
    
    np_obj.publish()
    print(f"✓ Published: {np_obj.source_uri}")
else:
    print("Publishing disabled. Set PUBLISH = True to enable.")

Loaded profile: Anne Fouilloux
✓ Signed
✓ Saved: quantum-biodiversity-search-strategy.signed.trig
✓ Published: https://w3id.org/np/RAEK3jctU2x3IKW174OTgmFH9zDygPiaD-vb4zGrD39A4


---
# 📋 JSON TEMPLATE

Create a JSON file with this structure:

```json
{
  "author": {
    "orcid": "0000-0000-0000-0000",
    "name": "Your Name"
  },
  "search_strategy": {
    "label": "Your search strategy title",
    "part_of": "https://w3id.org/np/RA...",
    "databases": [
      "https://pubmed.ncbi.nlm.nih.gov/",
      "https://www.scopus.com/"
    ],
    "search_terms": [
      "http://purl.obolibrary.org/obo/NCIT_C12345"
    ],
    "time_period": {
      "start": "2020-01-01",
      "end": "2025-12-01"
    },
    "languages": [
      "https://www.wikidata.org/wiki/Q1860"
    ],
    "methodology_notes": "Boolean operators and exclusion criteria..."
  },
  "output": {
    "filename": "my-search-strategy"
  }
}
```

## Common Database URIs

| Database | URI |
|----------|-----|
| PubMed | `https://pubmed.ncbi.nlm.nih.gov/` |
| Scopus | `https://www.scopus.com/` |
| Web of Science | `https://www.webofscience.com/` |
| arXiv | `https://arxiv.org/` |
| Google Scholar | `https://scholar.google.com/` |
| IEEE Xplore | `https://ieeexplore.ieee.org/` |
| Cochrane Library | `https://www.cochranelibrary.com/` |

## Common Language URIs (Wikidata)

| Language | URI |
|----------|-----|
| English | `https://www.wikidata.org/wiki/Q1860` |
| Portuguese | `https://www.wikidata.org/wiki/Q5146` |
| Spanish | `https://www.wikidata.org/wiki/Q1321` |
| French | `https://www.wikidata.org/wiki/Q150` |
| German | `https://www.wikidata.org/wiki/Q188` |

---